## MobileNet v1
- Paper : MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications
- (구글)모바일 환경에서도 사용할 수 있는 가벼운 딥뉴럴넷 만들기 위한 목적
- 기존 모바일 기기의 딥러닝 서비스를 Cloud Computing -> On Device화 (edge computing)
- AlexNet을 넘나드는 정확도를 보여주며, 경량화 model중 reference가 많이 되는 매우 유명한 모델
- 적은 연산량과 파라미터를 가지도록 설계, 
- **왜 기존 Conv인 입력 데이터에 필터 적용시, 모든 채널을 한번에 계산해서 아웃풋을 만들어야 할까가 연구의 시작 -> Depthwise Separable Convolution**

## Small Deep Neural Network 기법
- 1. Channel Reduction
    - 단순 Channel 감소 (Filter 감소)
- 2. Depthwise Separable Convolution
    - Mobilenet적용 및 Xception 에서 차용
- 3. Distillation & Compression
    - Knowledge Distillation & Network Compression
- 4. Remove Fully-Connected Layers
    - 파라미터의 대다수가 FC Layer에 존재 -> FC Layer 제거 필요
- 5. Kernel Reduction
    - 3x3 filter -> 1x1 filter (ex. SqueezeNet)
- 6. Evenly Spaced Downsampling
    - Downsampling을 언제 할 것인가에 대한 기법
    - Downsampling을 초반에 많이 할것인지 vs 후반에 많이 할것인지
    - 초반에 다운 샘플링을 많이 하는 경우
        - 네트워크의 크기는 줄게 되지만 Feature를 많이 잃어 버려 accuracy 감소
    - 후반에 다운 샘플링을 많이 하는 경우
        - accuracy는 전자보다 낫지만 네트워크의 크기가 많이 줄지는 않음
    - Evenly Spaced Downsampling은 해당 두 방법의 절충안으로 적절히 튜닝하며 균등하게 하자는 컨셉

- 7. Shuffle Operation
    - Maybe.. shufflenet?
<hr>


## 기존 Conv Operation
- Input Channel:3 - Channel이 3개(RGB)
- Input Filters:2 - 필터가 2장
- Output channel:2 - 최종 채널 (output)

![conv_operation](img/conv_operation.png)

- **부연설명** 
    - 27칸의 박스가 채널별 이미지(RGB)각 각각 3x3필터에 연산되고, 값을 더해서 최종 1칸의 값으로 모두 더해짐 (element-wise)

- 인풋 채널수 = 필터의 채널 수
    - 인풋의 채널이 3개이면 convolution연산시의 필터의 채널 또한 3개 (흑백=흑백필터, 컬러=컬러필터)
- 필터의 갯수 = output channel 갯수
    - 필터가 2개면, output channel 또한 2개
    
<hr>

## Standard, Depthwise, Pointwise Conv 차이점
- **Standard convolution , Depthwise Convolution, Pointwise Convolution**

![mobilenet_standard_depth_point](img/mobilenet_standard_depth_point.png)


![standardconv_vs_depthwise_conv](img/standardconv_vs_depthwise_conv.png)
    
<hr>

#### Layer 구성의 차이점
- 기존 Conv 3x3 Conv → BN → ReLu
- Xception 은 non-linearity relu 를 사이에 안씀.
- 모바일넷에서는 3 x 3 depthwise convolution → BN → ReLu → 1 x 1 convolution → BN → ReLU

![standardconv_vs_depthwise_conv_layer](img/standardconv_vs_depthwise_conv_layer.png)

<hr>

## MobileNet v1 주요 아이디어 (1) 

### Depthwise Separable Convolution
- **레이어에서 filtering 부분(depthwise) + combining 하는 부분(pointwise)을 지칭**
- 이러한 Factorization (분해) 방법은 Computation과 모델 사이즈 모두를 줄일수 있음
- 처음부터 Depthwise -> Pointwise(=1x1 conv)순으로 진행
- 순서
    - **(1) Depthwise Conv를 통해 각 channel의 정보를 유지하며 channel별로 Conv를 진행**
    - **(2) Pointwise Conv를 사용하여 각 channel간의 정보를 linear combination**
    - **(3) 결론적으로 기존의 3x3 Conv의 output 크기와 필요한 정보를 유지하면서 연산량은 줄인 것 -> 연산 속도의 증가**

- Depthwise Convolution
    - 채널을 각각 분리해서 Convolution 을 하는 것
    - 기존 Conv은 이를 모두 더하여 한 개의 출력값으로 만듬
    - Depthwise Convolution은 채널방향으로 합치지 않음

- Pointwise Convolution (= 1x1 conv 연산)
    - 채널정보를 합쳐서 Convolution 작업 ( Dimension reduction 효과)
    - 채널 방향의 연산을 진행

![depthwise_separable_conv](img/depthwise_separable_conv.png)

<hr>


## 연산속도 비교
- M, N 을 필터의 갯수, 차원의 갯수로 받아들여야 함

![operation_comparision](img/operation_comparision.PNG)

![mobilenet_math](img/mobilenet_math.PNG)

- 기본 Conv는 다 곱하기, But. Depthwise Separable Conv는 중간에 더하기 있음.
- 일반적으로 N과 Dk 중 아웃풋 채널인 N은 점점더 키워감 (32, 64, 128...) Dk는 필터의 크기로 보통 3
- 따라서 1/N과 1/Dk^2 에서 N은 큰 수니까 1/N은 매우 작은 수. 1/Dk^2 의 차이만 남는것


![depthwise_separable_conv_operation.png](img/depthwise_separable_conv_operation.png)

<hr>

## MobileNet Layer 구조 & Layer 별 비율

![mobilenet_layer](img/mobilenet_layer.png)

- Layer 구조
    - (224,224) 이미지를 받아 일반적인 Conv를 거치고, depthwise convolution과 pointwise convolution을 거치게 됨
    - 마지막에 Global Average Pooling과 FC layer 1개를 추가하여 Classification 구성

- Layer 별 비율
    - DW는 Depthwise Convolution S는 stride
    - 대다수의 연산과 파라미터가 1x1 conv 에 치중이 됨
    - 일반적인 Conv에서는 FC Layer에 연산과 파라미터가 치중되어 있는 것에서 변화함
    
<hr>

## MobileNet v1 주요 아이디어 (2) 
### Width Multiplier & Resolution multiplier
- 두 값 모두 기존의 컨셉에서 조금 더 작은 네트워크를 만들기 위한 scale 값 (0~1 사이 값)
- **Width Multiplier**
    - 논문에선 α 로 표현 
    - **인풋과 아웃풋의 채널에 곱해지는 값으로 채널의 크기를 일정비율 줄여가면서 실험**
    - 채널의 크기 M * α값(1, 0.75, 0.5, 0.25)

- **Resolution multiplier**
    - height와, width에 곱해지는 상수값. 
    - height와 weight가 Dk라면 pDk 가 됨
    - **상수 p는 (1, 0.857, 0.714, 0.571)이고 이는 (224, 192, 160, 128)의 이미지 크기로 실험**


## 실험 결과
- 즉 채널에 α, Feature Map에는 p가 곱해짐

- Table 4 
    - **depthwise separable conv가 기본 conv연산보다 정확도는 살짝 떨어지지만 네트워크 경량화에는 상당히 효율적인 것을 보여줌**
    
- Table 5, 6, 7
    - narrow(좁은)한 네트워크(네트워크의 height, width가 작은 것) vs shallow(얕은)한 네트워크( 네트워크의 깊이가 얕은 것) 중에 어떤 전략을 적절한가에 대한 실험
        - 실험의 결과를 보면 shallow한 것 보다 narrow한 것이 더 나음
        - **즉 네트워크 경량화를 해야 한다면 깊이를 줄이기 보다는 네트워크의 height, width를 줄이는 게 더 낫다는 것. 즉, 깊이 있게 쌓는 것이 더 낫다**

- **paramter의 경우 Width Multiplier가 더 많은 영향을 주고, Resolution Multiplier는 연산량에 많은 영향을 준다**
- 모바일넷은 정확도가 GoogLeNet이나 VGG16과 비슷한 수준까지 도달할 수 있음
- 유사한 성격의 경량화 모델인 스퀴즈넷과도 비교를 하였는데, 스퀴즈넷보다 파라미터 수는 약간 많지만 훨씬 적은 연산량으로 더 높은 정확도를 얻을 수 있음을 보여줌

![mobilenet_experiment](img/mobilenet_experiment.png)

- Detection 에서는 비교적 좋은 결과를 보이지 못했지만 효율적인 면에서 강조됨

## MobileNet 요약
- 모바일넷의 Depthwise Separable Conv는 Depthwise conv와 Pointwise Conv를 나눠서 하자는 것
- Xception에서 한 depthwise separable conv와는 조금 다르며 3 x 3 depthwise conv -> BN -> relu -> 1 x 1 conv -> BN -> relu 임
- Depthwise Conv : 채널을 각각 분리(RGB)해서 3x3Conv연산을 함
- Pointwise Conv : 채널정보를 합쳐서 1x1Conv 작업
- 연산량이 전체적으로 1/9가 줄어듬
- 차원(채널)수를 줄이는 Width Multiplier 실험과, 이미지 크기를 줄이는 Resolution Multiplier 실험 진행 => 차원수가 줄어드는 것이 더 성능저하를 불러왔음.
- Parameter를 줄이고 싶으면, 채널(차원)수를 줄여야 하고, 연산량을 줄이고 싶으면 Resolution Multiplier (이미지 크기)를 줄여야 함.


## 참고문헌
- https://www.youtube.com/watch?v=7UoOFKcyIvM&feature=youtu.be // PR-044
- https://gaussian37.github.io/dl-concept-mobilenet/
- https://hwangtoemat.github.io/paper-review/2020-03-20-MobileNets%EB%82%B4%EC%9A%A9/